In [2]:
import boto3
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.session import AWSSession
from rasterio.enums import Resampling


In [8]:
session = AWSSession(boto3.Session())


In [9]:
image_url = "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_MSIL2A_20210205T040621_N0214_R119_T46RBS_20210205T071926/cogs/TCI.tif"


In [23]:
import rasterio
from rasterio.session import AWSSession
from rasterio import warp

image_url = "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_MSIL2A_20230225T102021_N0509_R065_T32TMT_20230225T163245/cogs/TCI.tif"
image_url = "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/32/T/MT/2023/2/S2B_32TMT_20230220_0_L2A/TCI.tif"
with rasterio.Env(
    AWS_NO_SIGN_REQUEST="YES",
    AWS_REGIONAL_ENDPOINTS="FALSE",
    GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
):
    with rasterio.open(image_url, session=AWSSession()) as src:
        profile = src.profile
        # Create a new profile with the "tiled" and "blockxsize" options set
        profile.update(
            tiled=True,
            blockxsize=256,
            blockysize=256,
            compress="deflate",
            predictor=2,
        )
        # Define the output filename and open the output file for writing
        output_file = "output.tif"
        with rasterio.open(output_file, "w", **profile) as dst:
            # Copy the input data to the output file, block by block
            for ji, window in src.block_windows():
                data = src.read(window=window)
                dst.write(data, window=window)
            # Add overviews to the output file
            dst.build_overviews([2, 4, 8, 16])
